In [ ]:
# Manejo da data dos investimentos

from datetime import date

def transform_date_string_on_number_list(date_str):
    date_list = date_str.split('-')

    for index, element in enumerate(date_list):
        date_list[index] = int(date_list[index])
    
    return date_list

START_DATE = '2026-01-01'
TODAY_DATE = str(date.today())
start_year, start_month, start_day = transform_date_string_on_number_list(START_DATE)
today_year, today_month, today_day = transform_date_string_on_number_list(TODAY_DATE)


In [6]:
# Criar pastas para os arquivos

import os 

dir = os.path.abspath('../') + './archives'
dir_zip = os.path.abspath(dir) + './archives-zip'
dir_csv = os.path.abspath(dir) + './archives-csv'

if not os.path.exists(dir):
    os.mkdir(dir)

if not os.path.exists(dir_zip):
    os.mkdir(dir_zip)

if not os.path.exists(dir_csv):
    os.mkdir(dir_csv)

In [ ]:
# Realizar download dos arquivos da CVM

from urllib.request import urlretrieve, HTTPError

def adjust_month (month):
    if month < 10:
        month = '0' + str(month)
    
    return month

def download_inf_diario_archives (year, month):
    link = 'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_' + str(year) + month + '.zip'
    urlretrieve(link, '../archives/archives-zip/' + str(year) + str(month) + '.zip')

# Precisa do + 1 pois o loop em python só vale para a diferença entre números
for year in range (start_year, today_year + 1):
    
    MONTHS_START_YEAR, MONTHS_END_YEAR = 1, 12
    month_start_loop, month_end_loop = '', ''

    if today_year > year:
        month_start_loop = MONTHS_START_YEAR
        month_end_loop = MONTHS_END_YEAR + 1
    else:
        month_start_loop = MONTHS_START_YEAR
        month_end_loop = today_month + 1

    for month in range (month_start_loop, month_end_loop):
        month = adjust_month(month)

        try:
            download_inf_diario_archives(year, '01')
        except HTTPError as err: 
            if err.code == 404:
                print('Link não encontrado, ainda não foi disponibilizado o arquivo do mês ' + month + 'e do ano ' + year)
            else:
                print(err.msg)

In [8]:
# Unzip arquivos

from shutil import unpack_archive

archive_zip_dir = os.path.abspath('../archives/archives-zip')
archive_csv_dir = os.path.abspath('../archives/archives-csv/')
archives_list = os.listdir(archive_zip_dir)

ARCHIVE_TEMPLATE = 'inf_diario_fi_'
ARCHIVE_EXTENSION = '.csv'

for archive_zip_name in archives_list:
    archive_date = archive_zip_name[0:6]
    archive_csv_name = ARCHIVE_TEMPLATE + archive_date + ARCHIVE_EXTENSION

    archive_csv_search = os.path.join(archive_csv_dir, archive_csv_name)
    archive_zip_search = os.path.join(archive_zip_dir, archive_zip_name)

if not os.path.exists(archive_csv_search):
    unpack_archive(archive_zip_search, archive_csv_dir)
